# Preparation

In [1]:
# Add the parent directory of the current working directory to the Python path at runtime. 
# In order to import modules from the src directory.
import os
import sys 

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [33]:
# Import the required packages.
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from src.stat_utils import *

In [17]:
# Define the path to the data files.
data_path = '../data/empirical'
codebook_file = os.path.join(data_path, 'codebook.md')
data_file = os.path.join(data_path, 'main.csv')
metadata_file = os.path.join(data_path, 'meta.csv')

In [78]:
# Load codebook.
with open(codebook_file, 'r') as file:
    codebook_contents = file.read()

In [76]:
# load metadata and data.
metadata = pd.read_csv(metadata_file)
experimental_data = pd.read_csv(data_file)

# Variable Overview

In [79]:
display(Markdown(codebook_contents))

## Meta Data

### SubjectID
- **Description:** A unique identifier for each participant in the study.
- **Data Type:** Numeric.
- **Role:** Identifier.

### Age
- **Description:** The age of the participant at the time of study enrollment.
- **Data Type:** Numeric.
- **Role:** Attribute.

### Sex
- **Description:** The sex of the participant.
- **Data Type:** Categorical (F, M).
- **Role:** Attribute.


## Experimental Data

### SubjectID
- **Description:** A unique identifier for each participant in the study.
- **Data Type:** Numeric.
- **Role:** Identifier.

### SessionID
- **Description:** A unique identifier for each experimental session.
- **Data Type:** Numeric.
- **Role:** Identifier.

### BlockID
- **Description:** A unique identifier for each block of trials within a session.
- **Data Type:** Numeric.
- **Role:** Identifier.

### Condition
- **Description:** Represents one of the 25 unique stimulus conditions defined by contrast heterogeneity and grid coarseness.
- **Data Type:** Categorical (1 to 25).
- **Role:** Identifier.

### ContrastHeterogeneity
- **Description:** The range of contrast within the rectangular figure region; one of five values (0.01, 0.2575, 0.505, 0.7525, 1).
- **Data Type:** Numeric (Continuous).
- **Role:** Independent Variable.

### GridCoarseness
- **Description:** The scaling factor determining the coarseness of the grid; one of five values (1, 1.125, 1.250, 1.375, 1.5).
- **Data Type:** Numeric (Continuous).
- **Role:** Independent Variable.

### IndicatedOrientation
- **Description:** The orientation (horizontal or vertical) indicated by the participant for the rectangular region.
- **Data Type:** Categorical (Horizontal, Vertical).
- **Role:** Instrumental Variable.

### ActualOrientation
- **Description:** The actual orientation of the rectangular region within the stimulus.
- **Data Type:** Categorical (0 = Horizontal, 1 = Vertical).
- **Role:** Instrumental Variable.

### Correct
- **Description:** A binary variable indicating whether the participant's indicated orientation matches the actual orientation of the figure.
- **Data Type:** Binary (0, 1).
- **Role:** Dependent Variable.


# Hypotheses

**Hypothesis 1**: Discrimination accuracy in a figure-ground segregation task will vary systematically with changes in contrast heterogeneity and grid coarseness.

**Hypothesis 2**: As participants undergo perceptual learning, their ability to segregate figure from ground will improve. Therefore, discrimination accuracy should increase over sessions.

#### Expected Main Effects
- ContrastHeterogeneity
- GridCoarseness
- SessionID

#### Covariates
- SessionID x ContrastHeterogeneity
- SessionID x GridCoarseness

# Descriptives

In [71]:
# Provide sample information.
print(f'Number of participants: {len(metadata)}')
print(f'Number of females: {metadata["Sex"].value_counts().iloc[0]}')
print(f'Mean age: {metadata["Age"].mean():.2f}')

Number of participants: 8
Number of females: 6
Mean age: 23.75


# GEE Analyses

In [75]:
# define distribution and covariance structure for GEE
family = sm.families.Binomial()
covariance_structure = sm.cov_struct.Autoregressive(grid=True)

#### Full Model

In [ ]:
# ignore transfer (final) session
data = experimental_data[experimental_data['SessionID'] != 9]

In [77]:
# fit model and print results
model = smf.gee("Correct ~ ContrastHeterogeneity + GridCoarseness + SessionID + SessionID*ContrastHeterogeneity + SessionID*GridCoarseness", "SubjectID", data, cov_struct=covariance_structure, family=family)
results = model.fit()

print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                48000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                6000
                      Estimating Equations   Max. cluster size:                6000
Family:                           Binomial   Mean cluster size:              6000.0
Dependence structure:       Autoregressive   Num. iterations:                     5
Date:                     Wed, 24 Jan 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         10:56:31
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           2.9204  

#### Simple Effects

##### Session 1

In [80]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 1]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     4
Date:                     Wed, 24 Jan 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         15:03:40
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 1.3686      0.264      5.178      0.00

##### Session 2

In [ ]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 2]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     5
Date:                     Thu, 09 Nov 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         17:28:45
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 1.9408      0.336      5.783      0.00

##### Session 3

In [ ]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 3]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     4
Date:                     Thu, 09 Nov 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         17:28:45
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.1601      0.301      7.179      0.00

##### Session 4

In [ ]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 4]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     4
Date:                     Thu, 09 Nov 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         17:28:45
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.1708      0.337      6.444      0.00

##### Session 5

In [ ]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 5]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     4
Date:                     Thu, 09 Nov 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         17:28:45
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.4108      0.456      5.287      0.00

##### Session 6

In [ ]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 6]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     5
Date:                     Thu, 09 Nov 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         17:28:46
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.6448      0.402      6.585      0.00

##### Session 7

In [ ]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 7]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     4
Date:                     Thu, 09 Nov 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         17:28:46
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.5531      0.283      9.021      0.00

##### Session 8

In [ ]:
# select session
session_data = experimental_data[experimental_data['SessionID'] == 8]

# fit model
model = smf.gee("Correct ~ ContrastHeterogeneity", "SubjectID", session_data, cov_struct=covariance_structure, family=family)
results = model.fit()

# print results
print(results.summary())
print('\n\n')
print_wald_chi_square(results)

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:       Autoregressive   Num. iterations:                     4
Date:                     Thu, 09 Nov 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         17:28:46
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.8654      0.364      7.881      0.00

In [ ]:
session_data = experimental_data[experimental_data['SessionID'] == 1]
accuracy = session_data.groupby(['SubjectID']).mean()['Correct']
equivalence_bound = bootstrap(accuracy, num_repeats=100000, percentile=95, abs_diff=True)

0.06450000000000002
